# TheRefrigeration Cycle Extension with the New Components



## 1 A Home Heat Pump for Space Heating
The Cycle of [A Home Heat Pump for Space Heating](https://www.ohio.edu/mechanical/thermo/Intro/Chapt.1_6/refrigerator/heatpump.html)

The heat pump system absorbs heat from the evaporator placed outside in order to pump heat into the air flowing through the insulated duct over the condenser section

![ohio_48](./img/vcr/ohio_48.jpg)

In this example, we evaluate the following:

**The Heat Pump**

* Heat absorbed by the evaporator: Qin

* Heat rejected by the condenser: Qout

* Coefficient of Performance as a heat pump:COP_hp

* Determine the **mass** flow rate of the refrigerant R134a

**The Insulated Duct**

* Determine the **mass** flow rate of the air flowing in the insulated duct

* Assuming that all this heat is absorbed by the air, determine the exit **temperature** of the air at station




## 2 The New Component

The Combined New Component:Condenser + Insulated duct

![cd_id](./img/vcr/cd_id.jpg)


![combined_cd_id](./img/vcr/combined_cd_id.jpg)


### 2.1 The Class CondenserInsulatedDcut

* Class AirPort

* Classs CondenserInsulatedDcut


In [ ]:
# %load ../../SimVCCE/playground/ncomponents/airport.py
"""
AirPort

"""
import CoolProp.CoolProp as cp

class AirPort:
    R = 8.314472/29
    cp = None

    def __init__(self, dictport):
        """ create the port object"""
        self.p = float(dictport['p'])
        self.t = None
        self.vdot = None
        self.mdot = None
        self.cp = None
        if ("t" in dictport):
            self.t = float(dictport['t'])
        if ("vdot" in dictport):
            self.vdot = float(dictport['vdot'])
            self.mdot = self.p*self.vdot / \
                (AirPort.R*(self.t+273.15)*60)  # PV = nRT
        if (self.p is not None and self.t is not None):
            self.cp = cp.PropsSI("CPMASS", "P", self.p*1000,
                                 "T", self.t+273.15, "AIR")/1000
            AirPort.cp = self.cp
    
    def __str__(self):
        result = "(p kPa,t °C): {:.1f} {:.1f}".format(self.p, self.t)
        return result


In [ ]:
# %load ../../SimVCCE/playground/ncomponents/condenser_insulated_duct.py
"""
The combined condenser and insulated duct

                         ↓   iPort refrigerant
                   ┌─────┼─────┐
                   │  ←┌─┼─┐→  │
      oPortAir   ← ┤ Q │ Z │ Q │← iPortAir
                   │  ←└─┼─┘→  │
                   └─────┼─────┘
                         ↓      oPort refrigerant
  
 port air :{"p":100.0,"t":15.0,"vdot":8.0}
     p - kPa   t -  °C  vdot - m^3/min   mdot - kg/s
"""

import CoolProp.CoolProp as cp
from components.port import Port
from .airport import *

class CondenserInsulatedDcut:

    energy = "QOUT"
    devtype = "CONDENSER_INSULATED_DUCT"

    def __init__(self, dictDev):
        """ Initializes the condenser """
        self.name = dictDev['name']
        self.iPort = [Port(dictDev['iPort'])]
        self.oPort = [Port(dictDev['oPort'])]
        self.iPortAir = AirPort(dictDev["iPortAir"])
        self.oPortAir = AirPort(dictDev["oPortAir"])
     
        # map the port's name(str) to the obj
        self.portdict = {
            "iPort": self.iPort,
            "oPort": self.oPort
        }

    def state(self):
        """ Isobaric """
        if self.oPort[0].p is not None:
           self.iPort[0].p = self.oPort[0].p
        elif self.iPort[0].p is not None:
           self.oPort[0].p = self.iPort[0].p

    def balance(self):
        """ mass and energy balance of the condenser  """
        # 
        if self.iPort[0].mdot is not None:
            self.oPort[0].mdot = self.iPort[0].mdot
        elif self.oPort[0].mdot is not None:
            self.iPort[0].mdot = self.oPort[0].mdot
        self.Qout = self.iPort[0].mdot*(self.iPort[0].h-self.oPort[0].h)
        # air flows
        self.oPortAir.t = self.iPortAir.t + self.Qout / \
            (self.iPortAir.mdot*AirPort.cp)


    def __str__(self):
        result = '\n' + self.name
        result += '\n' + " PORTS "+Port.title
        result += '\n' + " iPort " + self.iPort[0].__str__()
        result += '\n' + " oPort " + self.oPort[0].__str__()
        result += '\nQout(kW): \t{:>.2f}'.format(self.Qout)
        result += '\n' + "Insulated Duct(Air):"
        result += '\n\t' + "vdot(m^3/min): {:.2f}".format(self.iPortAir.vdot)
        result += '\n\t' + "mdot(kg/s): {:.4f}".format(self.iPortAir.mdot)
        result += '\n\t' + "inlet:"+self.iPortAir.__str__()
        result += '\n\t' + "outlet:"+self.oPortAir.__str__()
        return result

if __name__ == "__main__":
    dict_cdduct=    {
        "name": "Condenser_duct",
        "devtype": "CONDENSER_INSULATED_DUCTs",
        "iPort": {"p":1.0,"t":60.0,"mdot":0.0184},
        "oPort": {"p": 1.0, "t": 30},
        "iPortAir": {"p": 100.0, "t": 15.0, "vdot": 8.0},
        "oPortAir": {"p": 95.0}
    }
    cdduct = CondenserInsulatedDcut(dict_cdduct)
    cdduct.state()
    cdduct.balance()
    print(cdduct)



### 2.2 The JSON 

#### 2.2.1 The CondenserInsulatedDcut

```json
 {
            "name": "condenser_duct",
            "devtype": "CONDENSER_INSULATED_DUCT",
            "iPort": {},
            "oPort": {
                "t": 30,
                "p": 1.0
            },
            "iPortAir": {
                "p": 100.0,
                "t": 15.0,
                "vdot": 8.0
            },
            "oPortAir": {
                "p": 95.0
            }
        }
       

```

#### 2.2.2 The Heat pump Syetem


In [ ]:
# %load ../../SimVCCE/playground/jsonmodel/vchp_ohio_48_combined.json
{
    "name": "The_Ohio_home_heat_pump48",
    "refrigerant": "R134a",
    "components": [
        {
            "name": "Compressor",
            "devtype": "COMPRESSOR",
            "iPort": {},
            "oPort": {
                "t": 60,
                "p": 1.0
            },
            "ef": 1.0,
            "win": 1.0
        },
        {
            "name": "condenser_duct",
            "devtype": "CONDENSER_INSULATED_DUCT",
            "iPort": {},
            "oPort": {
                "t": 30,
                "p": 1.0
            },
            "iPortAir": {
                "p": 100.0,
                "t": 15.0,
                "vdot": 8.0
            },
            "oPortAir": {
                "p": 95.0
            }
        },
        {
            "name": "ExpansionValve",
            "devtype": "EXPANSIONVALVE",
            "iPort": {},
            "oPort": {}
        },
        {
            "name": "Evaporator",
            "devtype": "EVAPORATOR",
            "iPort": {},
            "oPort": {
                "p": 0.14,
                "x": 1.0
            }
        }
    ],
    "connectors": [
        [
            {
                "devname": "Compressor",
                "port": "oPort"
            },
            {
                "devname": "condenser_duct",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "condenser_duct",
                "port": "oPort"
            },
            {
                "devname": "ExpansionValve",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "ExpansionValve",
                "port": "oPort"
            },
            {
                "devname": "Evaporator",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "Evaporator",
                "port": "oPort"
            },
            {
                "devname": "Compressor",
                "port": "iPort"
            }
        ]
    ]
}

##  3 Analysis of the Heat pump 

In [ ]:
# %load ../../SimVCCE/playground/demo_hp48_combined_comdenser_insulated_duct.py

"""
General Object-oriented Abstraction of VC Cycle 

The heat pump system:

* heat -> evaporator

* condenser -> heat -> the air flowing through the insulated duct

Author: Cheng Maohua cmh@seu.edu.cn
"""

import sys
import os
curpath = os.path.abspath(os.path.dirname(__file__))
sys.path.append(curpath+'/../vccpython/')

from vcc.utils import OutFiles, create_dictcycle_from_jsonfile
from vcc.vccobj import VCCycle
from components import compdict
from ncomponents.condenser_insulated_duct import CondenserInsulatedDcut


if __name__ == "__main__":
    # add new the components to compdict
    compdict["CONDENSER_INSULATED_DUCT"] = CondenserInsulatedDcut
    curpath = os.path.abspath(os.path.dirname(__file__))
    ResultFilePath = curpath+'/result/'

    # 1 VC heat pump 
    json_filename = curpath+'\\'+'./jsonmodel/vchp_ohio_48_combined.json'
 
    thedictcycle = create_dictcycle_from_jsonfile(json_filename)
    cycle = VCCycle(thedictcycle)
    cycle.simulator()
    OutFiles(cycle)
    ResultFileName = ResultFilePath+thedictcycle['name']
    OutFiles(cycle, ResultFileName + '.txt')   